Testing aws api here and LLM


In [1]:
!pip install boto3 matplotlib

In [2]:
import boto3

# Bedrock client
bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"  # or your region
)

In [9]:
# Simulated OCR output of student homework
homework_text = """
Describe an event which you did not look forward to attending but

turned out to exciting.

As a young teenager growing up in Singapore, I often found myself dreading certain
events or activities that my family and friends would plan. Whether it was a family
gathering, a school field trip, or a friend's birthday party, there were times when I just
did not feel like going. However, one particular event stands out in my memory as an
occasion where I surprised myself by having an unexpectedly great time.

It was a warm Saturday afternoon, and my mom had told me that we were going to
visit my great aunt who lived in a different part of the city. I was not particularly close
to my great aunt, and I did not really know what to expect from the visit. In fact, I was
feeling a bit grumpy and resentful about having to spend my weekend doing
something that I was not interested in. As we made our way to her house, I kept
thinking about all the other things I could have been doing instead.

When we arrived, my great aunt greeted us with a warm smile and a big hug. She
was a kind and gentle woman, and I immediately felt at ease in her company. As we
settled into her living room, she offered us some snacks and began to tell us stories
about her life. I was surprised to find that I was actually interested in what she had to
say, and I listened attentively as she shared her experiences growing up in
Singapore during a different era.

After a while, my greatest of aunt suggested that we take a walks around her
neighbourhood. I wasn't not sure what to expect, but I agreed to join her and my mom.
As we walked, my great aunt pointed out different landmarks and shared more
stories about her past. I found myself enjoying the walk, taking in the sights and

sounds of the neighborhood, and feeling grateful to be spending time with my family.
"""

In [ ]:
import json

#li loong is the lil bro

prompt_classify = f"""
<role>
You are an English tutor. 
A student submitted the following homework:
</role>

<homework>{homework_text}</homework>

<instructions>
Analyze the student's work and identify flaws in the homework regarding each of these categories, envelope within <analysis> tags:
- Grammar
- Vocabulary
- Writing
- Spelling
- Punctuation

Provide a strength index (1 to 5), envelope within <attribute> tags:
<example>
Grammar: 5
Vocabulary: 3
Writing: 2
Spelling: 1
Punctuation: 4
</example>
</instructions>
"""

response = bedrock.invoke_model(
    modelId="anthropic.claude-v2:1",
    body=json.dumps({
        "prompt": f"\n\nHuman: {prompt_classify}\n\nAssistant:",
        "max_tokens_to_sample": 300,
        "temperature": 0.3,
        "stop_sequences": ["\n\nHuman:"]
    })
)

result = json.loads(response["body"].read())
weakness_summary = result["completion"]
print("Weakness Classification:\n", weakness_summary)

Weakness Classification:
  <analysis>
Grammar: There are some grammatical errors such as incorrect verb tenses ("I wasn't not sure" instead of "I was not sure") and issues with subject-verb agreement ("my greatest of aunt" instead of "my great aunt").

Vocabulary: The vocabulary seems appropriate for the most part. No major issues.

Writing: The writing flows reasonably well, with a clear introduction, body, and conclusion. The sequence of events is logical. Some awkward phrasings could be improved.  

Spelling: There are a few spelling mistakes such as "greatest" instead of "great aunt" and "neighbourhood" instead of "neighborhood".

Punctuation: Occasional punctuation errors, like missing commas after introductory phrases ("As we walked"). But overall decent punctuation.
</analysis>

<attribute> 
Grammar: 3
Vocabulary: 4 
Writing: 4
Spelling: 3
Punctuation: 4
</attribute>


In [44]:
import sys
import os
sys.path.append(os.path.abspath("../utils"))

from generate_radar_chart import generate_radar_chart

generate_radar_chart(weakness_summary)


TypeError: generate_radar_chart() missing 1 required positional argument: 'scores'

In [ ]:
prompt_exercises = f"""
Create 3 short exercises for a student with the following weaknesses:

{weakness_summary}

Format as:
1. Exercise: ...
   Answer: ...
2. Exercise: ...
   Answer: ...
3. Exercise: ...
   Answer: ...
"""

response2 = bedrock.invoke_model(
    modelId="anthropic.claude-v2:1",
    body=json.dumps({
        "prompt": f"\n\nHuman: {prompt_exercises}\n\nAssistant:",
        "max_tokens_to_sample": 500,
        "temperature": 0.7,
        "stop_sequences": ["\n\nHuman:"]
    })
)

result2 = json.loads(response2["body"].read())
exercises = result2["completion"]
print("Generated Exercises:\n", exercises)
